In [1]:
import os as o
import joblib as jb
from tqdm import tqdm
from keras.applications import VGG16
from keras.preprocessing.image import load_img, img_to_array
import cv2
import numpy as np
from keras.applications.vgg16 import preprocess_input
import numpy as np
from mtcnn import MTCNN
from sklearn.metrics.pairwise import cosine_similarity
from PIL import Image

In [5]:
characters=o.listdir('cropped_faces')
characters

['Android 18',
 'Armin Arlert',
 'Boa Hancock',
 'Eren Yeager',
 'Goku',
 'Hinata Hyuga',
 'Historia Reiss',
 'Inosuke Hashibira',
 'Kakashi Hatake',
 'Katsuhiko Teshigawara',
 'Levi Ackerman',
 'Megumi Fushiguro',
 'Mikasa Ackerman',
 'Monkey D. Luffy',
 'Nami',
 'Naruto Uzumaki',
 'Nezuko Kamado',
 'Nico Robin',
 'Roronoa Zoro',
 'Saitama',
 'Sakura Haruno',
 'Sanji',
 'Sasha Blouse',
 'Sasuke Uchiha',
 'Satoru Gojo',
 'Spike Spiegel',
 'Taki Tachibana',
 'Tsunade',
 'Vegeta',
 'Yuji Itadori',
 'Zenitsu Agatsuma']

In [9]:
all_img = []
for i in characters:
    for j in o.listdir(o.path.join('cropped_faces', i)):
        all_img.append(o.path.join('cropped_faces', i, j))


In [10]:
len(all_img)

3720

In [11]:
jb.dump(all_img,open('all_imag.jbl','wb'))

In [12]:
face_model=jb.load('all_imag.jbl','rb')

In [13]:
model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3), pooling='avg')

In [14]:
print(model.summary())

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

None


In [15]:
def image_det(img_path, model):
    try:     
        if not o.path.exists(img_path):
            print(f"File does not exist: {img_path}")
            return None
        img = cv2.imread(img_path)
        if img is None:
            print(f"Error loading image {img_path}: Image is None")
            return None        
        img = cv2.resize(img, (224, 224))        
        array_of_image = np.array(img, dtype=np.float32)        
        expand_img = np.expand_dims(array_of_image, axis=0)    
        preproces_img = preprocess_input(expand_img)
        result = model.predict(preproces_img).flatten()
        return result
    except Exception as e:
        print(f"General error processing image {img_path}: {e}")
        return None

In [16]:
attribute = []

for file in tqdm(face_model):
    result = image_det(file, model)
    if result is not None:
        attribute.append(result)


  0%|          | 0/3720 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 264ms/step


  0%|          | 1/3720 [00:00<22:13,  2.79it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step


  0%|          | 2/3720 [00:00<15:26,  4.01it/s]

...

100%|█████████▉| 3718/3720 [13:04<00:00,  4.51it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step


100%|█████████▉| 3719/3720 [13:04<00:00,  4.69it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step


100%|██████████| 3720/3720 [13:04<00:00,  4.74it/s]


In [87]:
print(len(attribute))
jb.dump(attribute,('attribute.jbl'))

3720


['attribute.jbl']

In [88]:
attribute_list=np.array(jb.load(open("attribute.jbl",'rb')))

In [119]:
face = cv2.imread("test_1.jpg")

In [120]:
image = Image.fromarray(face)
image = image.resize((224,224))

In [121]:
face_array=np.asarray(image)
face_array=face_array.astype('float32')

In [122]:
expanded_image=np.expand_dims(face_array,axis=0)
preprocessed_image=preprocess_input(expanded_image)

In [123]:
result=model.predict(preprocessed_image).flatten()
result

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step


array([3.16294289e+00, 4.07526302e+00, 1.65202528e-01, 9.86671448e-01,
       3.25524187e+00, 0.00000000e+00, 3.12612534e-01, 1.21714127e+00,
       5.33101916e-01, 1.31633224e+01, 1.04274535e+00, 1.01084166e+01,
       2.58460355e+00, 1.41044052e-06, 1.98418961e+01, 1.25732923e+00,
       1.53937355e-01, 1.78428590e+00, 0.00000000e+00, 3.72997165e+00,
       0.00000000e+00, 4.81481266e+00, 1.72500362e+01, 1.67309785e+00,
       3.84540677e-01, 9.34678972e-01, 1.57151403e+01, 1.58573854e+00,
       1.84970713e+00, 6.04892635e+00, 1.75452724e-01, 2.61155939e+00,
       7.67110443e+00, 6.75251007e-01, 0.00000000e+00, 3.13092172e-01,
       0.00000000e+00, 4.88089228e+00, 0.00000000e+00, 4.28135109e+00,
       3.48647165e+00, 2.37429166e+00, 3.41052675e+00, 1.46261442e+00,
       8.17776299e+00, 6.51314020e+00, 6.63610876e-01, 7.93695402e+00,
       6.95263982e-01, 1.67560234e+01, 0.00000000e+00, 3.82393098e+00,
       3.87362766e+00, 9.23057556e+00, 1.36027622e+00, 4.94807959e-01,
      

In [124]:
result.shape

(512,)

In [125]:
cosine_similarity(result.reshape(1,-1),attribute_list[0].reshape(1,-1))[0][0]

np.float32(0.61280036)

In [126]:
simliarity=[]
for i in range (len(attribute_list)):
    sim_score=cosine_similarity(result.reshape(1,-1),attribute_list[i].reshape(1,-1))[0][0]
    simliarity.append(sim_score)

image_positions = sorted(list(enumerate(simliarity)),reverse=True,key=lambda x: x[1])[:3]

In [127]:
image_positions

[(2030, np.float32(0.99286926)),
 (2031, np.float32(0.99286926)),
 (2032, np.float32(0.90412855))]

In [128]:
for idx in image_positions:
    temp_img=cv2.imread(all_img[idx[0]])
    cv2.imshow('output',temp_img)
    cv2.waitKey(0)